# Тестирование

Тестировать:
+ тесты проверяют корректность кода
+ и позволяют бесстрашно изменять код даже в больших
проектах.

Не тестировать:
+ написание тестов требует времени,
+ нередко в проекте тестов больше чем кода,
+ работающие тесты не гарантируют корректность.

Тем не менее, ответ очевиден: конечно же тестировать!

## Модуль doctest

In [4]:
import doctest
import itertools
def rle(iterable):
    """Applies run-length encoding to an iterable.
    >>> list(rle(""))
    []
    >>> list(rle("mississippi"))
    [('m', 1), ('i', 1), ('s', 2), ('i', 1), ('s', 2), ('i', 1), ('p', 2), ('i', 1)]
    """
    for item, g in itertools.groupby(iterable):
        yield item, sum(1 for _ in g)


if __name__ == "__main__":
    doctest.testmod()

Директивы позволяют изменить то, как doctest сравнивает
ожидаемый вывод интерпретатора с фактическим.

• Например, директива NORMALIZE_WHITESPACE нормализует
пробельные символы перед сравнением:

list(rle("mississippi"))

... # doctest: +NORMALIZE_WHITESPACE

[('m', 1), ('i', 1), ('s', 2), ('i', 1),
('s', 2), ('i', 1), ('p', 2), ('i', 1)]

• А директива ELLIPSIS позволяет использовать символ ...,
который совпадает с любой строкой:

list(rle("mississippi"))

... # doctest: +ELLIPSIS

[('m', 1), ('i', 1), ('s', 2), ('i', 1), ...]


Модуль doctest позволяет проверить реализацию функции
на соответствие записанному сеансу интерпретатора.
 
Плюсы:

+ доступен в стандартной библиотеке,
+ решает задачу тестирования для небольших проектов,
+ доктесты их легко читать,
+ примеры кода в документации всегда актуальны.

Минусы:

+ доктесты требуют, чтобы у результата было содержательное
строковое представление,
+ длинные доктесты ухудшают читаемость документации,
+ нет способа запустить подмножество доктестов,
+ если в середине доктест

## assert

Напоминание:
+ оператор assert принимает два аргумента: условие и
произвольное значение,
+ если условие falsy, оператор поднимает исключение
AssertionError.

In [5]:
assert [], 42

AssertionError: 42

In [6]:
def test_rle():
    s = "mississippi"
    tmp = set(ch for ch, _count in rle(s))
    assert tmp == set(s[:-1] + s[1])
    assert not list(rle(""))

In [7]:
test_rle()

Хороший тест:
+ корректный,
+ понятный читателю,
+ конкретный, то есть проверяет что-то одно.

Попробуем улучшить тест для функции rle:

In [8]:
def test_rle():
    assert rle("mississippi") == [
    ('m', 1), ('i', 1), ('s', 2), ('i', 1),
    ('s', 2), ('i', 1), ('p', 2), ('i', 1)
    ]

In [9]:
def test_rle_empty():
    assert not list(rle(""))

In [10]:
test_rle_empty()

In [11]:
test_rle()

AssertionError: 

Оператор assert можно использовать для написания
тестов.

Плюсы:
+ тесты c assert легко читать,
+ они не используют ничего кроме стандартных средств
языка,
+ в отличие от доктестов это обычные функции.

Минусы:
+ запускать тесты нужно вручную,
+ их сложно отлаживать, потому что
+ для каждого типа утверждения приходится самостоятельно
конструировать сообщение об ошибке.


## Модуль unittest

Модуль unittest реализует функциональность JUnit для
тестирования кода на Python.

Наследие Java до сих пор в обилии присутствует в API.

Перепишем имеющиеся тесты с использованием unittest:

In [12]:
import unittest

class TestHomework(unittest.TestCase):
    def test_rle(self):
        self.assertEqual(rle("mississippi"), [...])

    def test_rle_empty(self):
        self.assertEqual(list(rle("")), [])

if __name__ == "__main__":
    unittest.main()

E
ERROR: C:\Users\Professional\AppData\Roaming\jupyter\runtime\kernel-d2d7875d-8091-4cbb-a3e4-401964a8b1e0 (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\Professional\AppData\Roaming\jupyter\runtime\kernel-d2d7875d-8091-4cbb-a3e4-401964a8b1e0'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

c:\users\professional\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Функция unittest.main загружает все тесты текущего
модуля и запускает их.

Тест — метод экземпляра unittest.TestCase,
начинающийся на test.

При необходимости тесты можно объединять в группы с
помощью класса unittest.TestSuite:
suite = unittest.TestSuite([
TestHomework(),
TestSomethingElse()
])

Указывать вручную, что нужно запустить довольно досадно.

Вопрос: можно ли лучше?

Ответ - можно! В python 3 можно указывать unittest где запускать тесты

In [ ]:
python -m unittest .

? aka fixtures — способ подготовить контекст, в котором
будут запускаться тесты.

Это можно использовать, например, для работы с
ресурсами: сокетами, файлами, временными
директориями.

Пример:

In [13]:
class TestHomeworkWithOracle(unittest.TestCase):
    def setUp(self):
        self.oracle = RleOracle("http://oracle.rle.com")
    
    def test_rle_against_oracle(self):
        s = "mississippi"
        self.assertEqual(list(rle(s)), self.oracle(s))
    
    def tearDown(self):
        self.oracule.close()

Модуль unittest — клон JUnit для Python.

Плюсы:
+ доступен в стандартной библиотеке,
+ выводит понятные сообщения об ошибках,
+ умеет автоматически находить тесты.

Минусы:
+ API унаследован от Java,
+ заставляет писать много лишнего кода,
+ читать unittest тесты сложнее, чем доктесты и тесты,
использующие assert.


## Пакет py.test

Пакет py.test — популярная альтернатива unittest для
написания и запуска тестов.

Отличительная особенность py.test — практически
полное отсутствие API: тесты мо

In [15]:
def test_rle():
    assert rle("mississippi") == [
    ('m', 1), ('i', 1), ('s', 2), ('i', 1),
    ('s', 2), ('i', 1), ('p', 2), ('i', 1)
    ]

def test_rle_empty():
    assert not list(rle(""))

Запустить py.test можно 1001 способом, например:
+ найти тесты в текущей директории и во всех вложенных
директориях и запустить их:

$ python -m pytest

+ найти и запустить тесты в указанном файле:

$ python -m pytest test_pytest.py

+ запустить один тест в файле по имени:

$ python -m pytest test_pytest.py::test_rle

Что такое тест для py.test?
+ функция test_*,
+ метод test_* в классе Test* или в классе, наследующемся
от unittest.TestCase,
+ доктест, если py.test был запущен с параметром
--doctest-modules.

https://pytest.org/latest/goodpractises.html

 Пакет py.test — швейцарский нож тестирования в мире
Python.

Плюсы:
+ практически нет API, тесты — обычные функции,
+ удобный вывод,
+ удобный механизм параметризации тестов,
+ приспособления, которые можно переиспользовать,
+ множество встроенных возможностей и впечатляющее
количество расширений (https://pytest.org/latest/plugins_index
)


Минусы:
+ магия-магия-магия,
+ может быть сложнее для понимания, если вы привыкли к
JUnit.


## Пакет hypothesis

• До текущего момента мы обсуждали тесты, которые
проверяют тривиальные свойства кода:

+ равенство ожидаемому значению,
+ вхождение результата в коллекцию и так далее.

Иногда можно формулировать и проверять менее
тривиальные свойства, например:
+ если sorted возвращает список, отсортированный по
неубыванию,
+ то для любого списка xs и индексов i < j справедливо,
что sorted(xs)[i] <= sorted(xs)[j].

Вопрос: как проверить это свойство?

In [16]:
import random

def random_array():
    size = random.randint(0, 1024)
    return [random.randint(-42, 42) for _ in range(size)]

def test_sort():
    xs = random_array()
    result = sorted(xs)
    assert all(xi <= xj
               for xi, xj in zip(result, result[1:]))

Такой подход сработает, но
+ писать генераторы самостоятельно долго и бессмысленно,
+ на практике нас, как правило, интересует минимальный
контрпример.

Пакет hypothesis реализует API для формулирования и
проверки свойств(гипотез).

Перепишем test_sort с использованием hypothesis:

In [18]:
import hypothesis.strategies as st
from hypothesis import given

@given(st.lists(st.integers()))
def test_sort(xs):
    result = sorted(xs)
    assert all(xi <= xj
               for xi, xj in zip(result, result[1:]))
# И попробуем обмануть его:

def sorted(xs, f=sorted):
    return xs if len(xs) == 8 else f(xs)

Пакет hypothesis позволяет удобно формулировать и
проверять свойства про Python код.

Львиная часть функциональности hypothesis осталась за
кадром:
+ как сузить пространство поиска контпримеров?
+ как написать свой генератор?
+ как формулировать зависимые гипотезы?